# Ajuste do circuito RLC com R = 300Ω

In [20]:
import pandas as pd
import altair as alt

### Leitura dos dados de tensão obtidos através do osciloscópio

In [21]:
p4b_data = pd.read_csv('./p4b_data0.csv')
p4b_data.head()

,Time(s),CH1V,CH2V
0,-0.002000,0.001867,-0.025867
1,-0.001998,-0.024800,0.084533
2,-0.001996,-0.006933,0.190667
3,-0.001994,-0.002667,0.225867
4,-0.001992,-0.029333,0.270133


Setando o tempo 0:

In [22]:
p4b_data["Time(s)"] = p4b_data["Time(s)"] + 2.000000e-03
p4b_data.head()

,Time(s),CH1V,CH2V
0,0.000000,0.001867,-0.025867
1,0.000002,-0.024800,0.084533
2,0.000004,-0.006933,0.190667
3,0.000006,-0.002667,0.225867
4,0.000008,-0.029333,0.270133


### Visualização inicial dos dados

In [29]:
p4b_CH1V = alt.Chart(p4b_data).mark_line(color='orange').encode(
    x = "Time(s)",
    y = "CH1V",
    tooltip = "Time(s)"
).interactive()

p4b_CH2V = alt.Chart(p4b_data).mark_point().encode(
    x = alt.X("Time(s)", title="Tempo (s)"),
    y = alt.Y("CH2V", title="Tensão (V)"),
    tooltip = "Time(s)"
).properties(title='Tensões medidas').interactive()

p4b_chart = p4b_CH2V + p4b_CH1V
p4b_chart

alt.LayerChart(...)

### Selecionando uma oscilação

In [24]:
p4b = p4b_data[(p4b_data["Time(s)"] >= 0.000464) & (p4b_data["Time(s)"] <= 0.000976)]
p4b_chart = alt.Chart(p4b).mark_point().encode(
    x = alt.X("Time(s)", scale=alt.Scale(domain=[0.000464, 0.000976])),
    y = alt.Y("CH2V"),
    tooltip = ["Time(s)", "CH2V"]
).interactive()
p4b_chart

alt.Chart(...)

### Configurando o fit para o ajuste 

In [25]:
import numpy as np
from scipy.optimize import curve_fit

# Função do modelo para o ajuste
def oscilacao_amortecida(t, Voff, A, alpha, wp, t0):
    return Voff + A * np.exp(-alpha * (t - t0)) * np.cos(wp * (t - t0))

### Ajustando a oscilação selecionada

In [30]:
# Dados experimentais
tempo = p4b['Time(s)'].to_numpy()
tensao_ruidosa = p4b['CH2V'].to_numpy()

# Ajuste dos parâmetros usando curve_fit
p0 = [0, 2.5, 39950, 104096, 0.000464]  # Estimativas iniciais para os parâmetros
params, covariance = curve_fit(oscilacao_amortecida, tempo, tensao_ruidosa, p0=p0)

# Parâmetros ajustados
Voff, A, alpha, wp, t0 = params
param_incertezas = np.sqrt(np.diag(covariance))
print(f"Parâmetros ajustados:")
print(f"Voff = {Voff:.4f} V | Incerteza: {param_incertezas[0]}")
print(f"A = {A:.4f} V | Incerteza: {param_incertezas[1]}")
print(f"alpha = {alpha:.4f} s^-1 | Incerteza: {param_incertezas[2]}")
print(f"wp = {wp:.4f} rad/s | Incerteza: {param_incertezas[3]}")
print(f"t0 = {t0:.4f} s | Incerteza: {param_incertezas[4]}")

# Calcular a frequência natural (omega_0) a partir de wp e alpha
omega_0 = np.sqrt(wp**2 + alpha**2)
print(f"Frequência natural (omega_0) = {omega_0:.4f} rad/s")

# Gerar dados ajustados para comparação
tensao_ajustada = oscilacao_amortecida(tempo, *params)

# Criar dataframe para visualização
df = pd.DataFrame({
    'Tempo (s)': tempo,
    'Tensão Medida (V)': tensao_ruidosa,
    'Tensão Ajustada (V)': tensao_ajustada,
    'nome_da_curva_2': 'Curva ajustada'
})

# Visualização com Altair
chart = alt.Chart(df).mark_point().encode(
    x='Tempo (s)',
    y=alt.Y('Tensão Medida (V)', title='Tensão (V)', axis=alt.Axis(grid=True)),
    tooltip=['Tempo (s)', 'Tensão Medida (V)']
).properties(title='Ajuste de Oscilação Amortecida').interactive()

ajuste_chart = alt.Chart(df).mark_line(color='red').encode(
    x='Tempo (s)',
    y='Tensão Ajustada (V)',
    color=alt.Color('nome_da_curva_2', legend=alt.Legend(title="", orient="top-right"), scale=alt.Scale(range=['red']))
)

ajuste_chart + chart


Parâmetros ajustados:
Voff = 4.9319 V | Incerteza: 0.0009912710916398812
A = -4.4023 V | Incerteza: 0.016274807548369578
alpha = 48386.7465 s^-1 | Incerteza: 181.24744573862003
wp = 108306.8822 rad/s | Incerteza: 280.3051639500986
t0 = 0.0005 s | Incerteza: 4.504776271732074e-08
Frequência natural (omega_0) = 118624.0193 rad/s


alt.LayerChart(...)